In [21]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = os.getenv("Langsmith_api")
os.environ["LANGSMITH_PROJECT"] = "flashcard_rag_backend"

In [23]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

CHROMA_dir = "./chroma_db"

embeddings_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

def get_vectorstore(deck_id):
    deck_path = os.path.join(CHROMA_dir, deck_id)
    os.makedirs(deck_path, exist_ok=True)
    return Chroma(persist_directory=deck_path, embedding_function=embeddings_function)
    

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def add_documents(deck_id: str, docs: list[str]):
    splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
    splits = []
    for doc in docs:
        splits.extend(splitter.split_text(doc))
    vectorstore = get_vectorstore(deck_id)
    vectorstore.add_texts(splits)

In [26]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:3B")

response = llm.invoke("What is the fullform of NLP?")
print(response.content)

The full form of NLP is Natural Language Processing.


In [2]:
# List all available local Ollama models (running in WSL)
import subprocess
import json

try:
    # Run ollama list command through WSL
    result = subprocess.run(['wsl', 'ollama', 'list'], capture_output=True, text=True, check=True)
    
    print("Available Ollama Models (from WSL):")
    print("=" * 50)
    print(result.stdout)
    
except subprocess.CalledProcessError as e:
    print(f"Error running ollama list through WSL: {e}")
    print("Make sure:")
    print("1. WSL is installed and working")
    print("2. Ollama is installed in WSL")
    print("3. Ollama service is running in WSL")
    print("\nTry running 'wsl ollama list' in PowerShell to test manually")
except FileNotFoundError:
    print("WSL command not found. Please make sure WSL is installed and in your PATH")
    print("Alternative: You can also try accessing Ollama via HTTP API on localhost:11434")
except Exception as e:
    print(f"Unexpected error: {e}")
    
# Alternative method: Try to access Ollama via HTTP API
print("\n" + "="*50)
print("Alternative: Checking Ollama via HTTP API...")
try:
    import requests
    response = requests.get('http://localhost:11434/api/tags', timeout=5)
    if response.status_code == 200:
        models_data = response.json()
        print("Available models via HTTP API:")
        for model in models_data.get('models', []):
            print(f"- {model.get('name', 'Unknown')} (Size: {model.get('size', 'Unknown')})")
    else:
        print(f"HTTP API returned status code: {response.status_code}")
except requests.exceptions.RequestException as e:
    print(f"Could not connect to Ollama HTTP API: {e}")
    print("Make sure Ollama is running and accessible on localhost:11434")
except ImportError:
    print("requests library not available for HTTP API check")

Error running ollama list through WSL: Command '['wsl', 'ollama', 'list']' returned non-zero exit status 127.
Make sure:
1. WSL is installed and working
2. Ollama is installed in WSL
3. Ollama service is running in WSL

Try running 'wsl ollama list' in PowerShell to test manually

Alternative: Checking Ollama via HTTP API...
Available models via HTTP API:
- deepseek-coder:1.3b-base-q4_K_M (Size: 873596816)
- gemma3n:latest (Size: 7547589116)
- llama3.2:3B (Size: 2019393189)
- gemma:2B (Size: 1678456656)
Available models via HTTP API:
- deepseek-coder:1.3b-base-q4_K_M (Size: 873596816)
- gemma3n:latest (Size: 7547589116)
- llama3.2:3B (Size: 2019393189)
- gemma:2B (Size: 1678456656)


In [27]:
from langchain.chains import RetrievalQA

def ask_question(deck_id: str, question: str):
    vectorstore = get_vectorstore(deck_id)
    retriever = vectorstore.as_retriever()
    qa = RetrievalQA.from_chain_type(
        llm = llm,
        retriever = retriever,
        return_source_documents = True
    )
    result = qa({"query": question})
    return {
        "answer": result["result"],
        "sources": [doc.page_content for doc in result["source_documents"]]
    }